بخش پایین برای لاگین کردن در اکانتی است که قرار است داده‌های سوال با فرمت گفته شده در درایو آن اکانت باشند. در صورتی که آدرس ذخیره داده با آدرس پیشفرض تفاوت دارد لطفا جهت کارکرد صحیح برنامه آدرس را به روز کنید. 

In [ ]:
from google.colab import drive
 drive.mount('/content/gdrive')

In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from tqdm import tqdm
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

بخش پایین مربوط به تعریف ترنسفورم‌های مذکور در گزارش است.

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

پخش پایین مربوط به لود کردن داده‌ها است.

In [ ]:
data_dir = "/content/gdrive/My Drive/CoputerVision/Data"
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
         for x in ['train', 'val']}
dst_loader = {x: torch.utils.data.DataLoader(dsets[x], batch_size=batch_size,
                                               shuffle=True, num_workers=2)
                for x in ['train', 'val']}
dset_sizes = {x: len(dsets[x]) for x in ['train', 'val']}
dset_classes = dsets['train'].classes


کد زیر مربوط به تابع آموزش و اعتبار سنجی است که در گزارش توضیح داده شده است

In [ ]:
def train_and_val_network(model, criterion, optimizer, scheduler, num_epochs):
  best_model = model;
  best_acc = 0.0;
  epochs_number = []
  train_loss_values = []
  val_loss_values = []
  train_acc_values = []
  val_acc_values = []
  train_acc_t5_values = []
  val_acc_t5_values = []
  for epoch in tqdm(range(num_epochs)):
    for phase in ['train', 'val']:
      if phase == 'train':
        scheduler.step()
        model.train() 
      else:
        model.eval()
      current_loss = 0.0
      current_corrects = 0
      current_corrects_t5 = 0
      for data in dst_loader[phase]:
        inputs, labels = data
        if torch.cuda.is_available():
          inputs = inputs.to('cuda')
          labels = labels.to('cuda')
        inputs, labels = Variable(inputs), Variable(labels)
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        _, predsP = outputs.topk(5, 1)
        loss = criterion(outputs, labels)
        if phase == 'train':
          loss.backward()
          optimizer.step()
        current_loss += loss.item()
        current_corrects += torch.sum(preds == labels.data)
        current_corrects_t5 += torch.sum(predsP == labels.view(-1, 1).expand_as(predsP))
      epoch_loss = current_loss / dset_sizes[phase]
      epoch_acc = current_corrects.item() / float(dset_sizes[phase])
      epoch_acc_t5 = current_corrects_t5.item() / float(dset_sizes[phase])
      epochs_number.append(epoch)
      if phase == 'train':
        train_loss_values.append(epoch_loss)
        train_acc_values.append(epoch_acc)
        train_acc_t5_values.append(epoch_acc_t5)
      else:
        if epoch_acc > best_acc:
          best_acc = epoch_acc
          best_model = copy.deepcopy(model)
        val_loss_values.append(epoch_loss)
        val_acc_values.append(epoch_acc)
        val_acc_t5_values.append(epoch_acc_t5)
  return best_model, train_loss_values, val_loss_values, train_acc_values, val_acc_values, train_acc_t5_values, val_acc_t5_values, best_acc


 پارامترها و کد اصلی و رسم نمودار مربوط به بخش 1

In [ ]:
# define parameters
num_epochs = 40
num_classes = 15
feature_extract = False
use_pretrained = False
input_size = 224
base_lr = 0.001
batch_size = 10
# main code
model_ft = models.alexnet(pretrained=use_pretrained)
model_ft.features[2] = nn.MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
model_ft.features = model_ft.features[:3]
model_ft.classifier[4] = nn.Linear(in_features=2304, out_features=4096, bias=True)
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
model_ft.classifier = model_ft.classifier[3:7]
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer_ft, gamma=0.9)
if torch.cuda.is_available():
  model_ft = model_ft.to('cuda')
  criterion = criterion.to('cuda')
model_ft, train_loss_values, val_loss_values, train_acc_values, val_acc_values, train_acc_t5_values, val_acc_t5_values, best_acc = train_and_val_network(model_ft, criterion, optimizer_ft, scheduler, num_epochs)
torch.save(model_ft, 'best_model_p1.pth')
print(best_acc)
# draw charts
from google.colab import files
plt.figure
plt.title("Accuracy vs. Number of Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.plot(range(1,num_epochs+1),train_acc_values, label="Train")
plt.plot(range(1,num_epochs+1),val_acc_values, label="Val")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.savefig('Acc_p1.png')
files.download('Acc_p1.png')
plt.clf()
plt.close()
plt.figure
plt.title("Accuracy(top 5) vs. Number of Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.plot(range(1,num_epochs+1),train_acc_t5_values, label="Train")
plt.plot(range(1,num_epochs+1),val_acc_t5_values, label="Val")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.savefig('AccT5_p1.png')
files.download('AccT5_p1.png')
plt.clf()
plt.close()
plt.figure()
plt.title("Loss vs. Number of Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(range(1,num_epochs+1),train_loss_values, label="Train")
plt.plot(range(1,num_epochs+1),val_loss_values, label="Val")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.savefig('Loss_p1.png')
files.download('Loss_p1.png')

 پارامترها و کد اصلی و رسم نمودار مربوط به بخش 2

In [ ]:
# define parameters
num_epochs = 40
num_classes = 15
feature_extract = False
use_pretrained = False
input_size = 224
base_lr = 0.001
batch_size = 32
# main code
model_ft = models.alexnet(pretrained=use_pretrained)
model_ft.features[6] = nn.Conv2d(192, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
model_ft.features[8] = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
model_ft.features = model_ft.features[:9]
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
criterion = nn.CrossEntropyLoss()
if torch.cuda.is_available():
  model_ft = model_ft.to('cuda')
  criterion = criterion.to('cuda')
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer_ft, gamma=0.9)

model_ft, train_loss_values, val_loss_values, train_acc_values, val_acc_values, train_acc_t5_values, val_acc_t5_values,  best_acc = train_and_val_network(model_ft, criterion, optimizer_ft, scheduler, num_epochs)
torch.save(model_ft, 'best_model_p3.pth')
print(best_acc)
#draw charts
from google.colab import files
plt.figure
plt.title("Accuracy vs. Number of Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.plot(range(1,num_epochs+1),train_acc_values, label="Train")
plt.plot(range(1,num_epochs+1),val_acc_values, label="Val")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.savefig('Acc_p2.png')
files.download('Acc_p2.png')
plt.clf()
plt.close()
plt.figure
plt.title("Accuracy(top 5) vs. Number of Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.plot(range(1,num_epochs+1),train_acc_t5_values, label="Train")
plt.plot(range(1,num_epochs+1),val_acc_t5_values, label="Val")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.savefig('AccT5_p2.png')
files.download('AccT5_p2.png')
plt.clf()
plt.close()
plt.figure()
plt.title("Loss vs. Number of Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(range(1,num_epochs+1),train_loss_values, label="Train")
plt.plot(range(1,num_epochs+1),val_loss_values, label="Val")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.savefig('Loss_p2.png')
files.download('Loss_p2.png')

 پارامترها و کد اصلی و رسم نمودار مربوط به بخش 3

In [ ]:
# define parameters
num_epochs = 40
num_classes = 15
feature_extract = False
use_pretrained = False
input_size = 224
base_lr = 0.001
batch_size = 32
# main code
model_ft = models.alexnet(pretrained=use_pretrained)
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
criterion = nn.CrossEntropyLoss()
if torch.cuda.is_available():
  model_ft = model_ft.to('cuda')
  criterion = criterion.to('cuda')
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer_ft, gamma=0.9)

model_ft, train_loss_values, val_loss_values, train_acc_values, val_acc_values, train_acc_t5_values, val_acc_t5_values, best_acc = train_and_val_network(model_ft, criterion, optimizer_ft, scheduler, num_epochs)
torch.save(model_ft, 'best_model_p3.pth')
print(best_acc)
# draw charts
from google.colab import files
plt.figure
plt.title("Accuracy vs. Number of Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.plot(range(1,num_epochs+1),train_acc_values, label="Train")
plt.plot(range(1,num_epochs+1),val_acc_values, label="Val")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.savefig('Acc_p3.png')
files.download('Acc_p3.png')
plt.clf()
plt.close()
plt.figure
plt.title("Accuracy(top 5) vs. Number of Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.plot(range(1,num_epochs+1),train_acc_t5_values, label="Train")
plt.plot(range(1,num_epochs+1),val_acc_t5_values, label="Val")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.savefig('AccT5_p3.png')
files.download('AccT5_p3.png')
plt.clf()
plt.close()
plt.figure()
plt.title("Loss vs. Number of Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(range(1,num_epochs+1),train_loss_values, label="Train")
plt.plot(range(1,num_epochs+1),val_loss_values, label="Val")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.savefig('Loss_p3.png')
files.download('Loss_p3.png')

 پارامترها و کد اصلی و رسم نمودار مربوط به بخش 4

In [ ]:
# define parameters
num_epochs = 30
num_classes = 15
feature_extract = True
use_pretrained = True
input_size = 224
base_lr = 0.001
batch_size = 32
# main code
model_ft = models.alexnet(pretrained=use_pretrained)
if feature_extract:
  for param in model_ft.parameters():
    param.requires_grad = False
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
criterion = nn.CrossEntropyLoss()
if torch.cuda.is_available():
  model_ft = model_ft.to('cuda')
  criterion = criterion.to('cuda')
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer_ft, gamma=0.9)

model_ft, train_loss_values, val_loss_values, train_acc_values, val_acc_values, train_acc_t5_values, val_acc_t5_values, best_acc = train_and_val_network(model_ft, criterion, optimizer_ft, scheduler, num_epochs)
torch.save(model_ft, 'best_model_p4.pth')
print(best_acc)
# draw charts
from google.colab import files
plt.figure
plt.title("Accuracy vs. Number of Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.plot(range(1,num_epochs+1),train_acc_values, label="Train")
plt.plot(range(1,num_epochs+1),val_acc_values, label="Val")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.savefig('Acc_p4.png')
files.download('Acc_p4.png')
plt.clf()
plt.close()
plt.figure
plt.title("Accuracy(top 5) vs. Number of Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.plot(range(1,num_epochs+1),train_acc_t5_values, label="Train")
plt.plot(range(1,num_epochs+1),val_acc_t5_values, label="Val")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.savefig('AccT5_p4.png')
files.download('AccT5_p4.png')
plt.clf()
plt.close()
plt.figure()
plt.title("Loss vs. Number of Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(range(1,num_epochs+1),train_loss_values, label="Train")
plt.plot(range(1,num_epochs+1),val_loss_values, label="Val")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.savefig('Loss_p4.png')
files.download('Loss_p4.png')

 پارامترها و کد اصلی و رسم نمودار مربوط به بخش 5

In [ ]:
# define parameters
num_epochs = 30
num_classes = 15
feature_extract = False
use_pretrained = True
input_size = 224
base_lr = 0.001
batch_size = 32
# main code
model_ft = models.alexnet(pretrained=use_pretrained)
if feature_extract:
  for param in model_ft.parameters():
    param.requires_grad = False
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
criterion = nn.CrossEntropyLoss()
if torch.cuda.is_available():
  model_ft = model_ft.to('cuda')
  criterion = criterion.to('cuda')
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer_ft, gamma=0.9)

model_ft, train_loss_values, val_loss_values, train_acc_values, val_acc_values, train_acc_t5_values, val_acc_t5_values, best_acc = train_and_val_network(model_ft, criterion, optimizer_ft, scheduler, num_epochs)
torch.save(model_ft, 'best_model_p5.pth')
print(best_acc)
# draw charts
from google.colab import files
plt.title("Accuracy vs. Number of Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.plot(range(1,num_epochs+1),train_acc_values, label="Train")
plt.plot(range(1,num_epochs+1),val_acc_values, label="Val")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.savefig('Acc_p5.png')
files.download('Acc_p5.png')
plt.clf()
plt.close()
plt.figure
plt.title("Accuracy(top 5) vs. Number of Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.plot(range(1,num_epochs+1),train_acc_t5_values, label="Train")
plt.plot(range(1,num_epochs+1),val_acc_t5_values, label="Val")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.savefig('AccT5_p5.png')
files.download('AccT5_p5.png')
plt.clf()
plt.close()
plt.title("Loss vs. Number of Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(range(1,num_epochs+1),train_loss_values, label="Train")
plt.plot(range(1,num_epochs+1),val_loss_values, label="Val")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.savefig('Loss_p5.png')
files.download('Loss_p5.png')